In [1]:
class Sentiment:
    Positive = "Positive"
    Negative = "Negative"
    Neutral = "Neutral"

class Review:
    def __init__(self,text,score):
        self.text = text
        self.score = score
        self.sentiment = self.getSentiment()
        
    def getSentiment(self):
        if (self.score <= 2):
            return Sentiment.Negative
        elif self.score == 3:
            return Sentiment.Neutral
        else:
            return Sentiment.Positive
        

f = spark.read.json('/home/shivam/intern/rev.json')

In [2]:
f.rdd.getNumPartitions()

8

In [3]:
f.rdd.glom().map(len).collect()

[73041, 72008, 71444, 69394, 70715, 73611, 73776, 67348]

In [4]:
f.count()

571337

In [5]:
f.groupBy("overall").count().orderBy("overall").show()

+-------+------+
|overall| count|
+-------+------+
|    1.0| 16789|
|    2.0| 19587|
|    3.0| 45818|
|    4.0|122647|
|    5.0|366496|
+-------+------+



In [6]:
df = f.withColumn("overint",f["overall"].cast("Int"))

In [7]:
#df.printSchema()

df.select('overint').show()

+-------+
|overint|
+-------+
|      5|
|      5|
|      4|
|      4|
|      4|
|      5|
|      5|
|      4|
|      5|
|      5|
|      4|
|      3|
|      5|
|      4|
|      5|
|      5|
|      5|
|      4|
|      4|
|      2|
+-------+
only showing top 20 rows



In [8]:
df2 = df.select("overint","reviewText")

In [66]:
#df2 = df2.repartition(5,'overint')
df2 = df2.coalesce(5)

In [67]:
df2.rdd.getNumPartitions()

5

In [68]:
df2.show()

+-------+--------------------+
|overint|          reviewText|
+-------+--------------------+
|      5|This came in on t...|
|      5|I had a factory G...|
|      4|If you don't have...|
|      4|This works no bet...|
|      4|I purchased this ...|
|      5|Needed this tool ...|
|      5|If u don't have i...|
|      4|This light will n...|
|      5|Light and laser t...|
|      5|Does everything i...|
|      4|Very bright.  I w...|
|      3|It's cheaply made...|
|      5|Mine arrived with...|
|      4|It works great it...|
|      5|I love this light...|
|      5|Bit bulky. One bu...|
|      5|it is bright and ...|
|      4|A mice bright lig...|
|      4|Had one ride on t...|
|      2|So it worked well...|
+-------+--------------------+
only showing top 20 rows



In [113]:
df2.groupBy("overint").count().orderBy("overint").show()

+-------+------+
|overint| count|
+-------+------+
|      1| 16789|
|      2| 19587|
|      3| 45818|
|      4|122647|
|      5|366496|
+-------+------+



In [69]:
df2.write.mode("overwrite").json("/home/shivam/intern/coal")

In [70]:
df2.rdd.glom().map(len).collect()

[73041, 143452, 69394, 144326, 141124]

In [87]:
import json

part0 = "/home/shivam/intern/coal/part-00004-d3495b0e-29c8-47d5-aef5-60a1b3fe5b06-c000.json"

reviews1 = []
with open(part0) as p0:
    for line in p0:
        review = json.loads(line)
        reviews1.append(Review(review['reviewText'],review['overint']))

In [11]:
import json

part1 = "/home/shivam/intern/repart/part-00001-06a43a4e-b6a6-45a0-a12d-0ee5869b345b-c000.json"

reviews2 = []
with open(part1) as p1:
    for line in p1:
        review = json.loads(line)
        reviews2.append(Review(review['reviewText'],review['overint']))

In [12]:
import json

part2 = "/home/shivam/intern/repart/part-00002-06a43a4e-b6a6-45a0-a12d-0ee5869b345b-c000.json"

reviews3 = []
with open(part2) as p2:
    for line in p2:
        review = json.loads(line)
        reviews3.append(Review(review['reviewText'],review['overint']))

In [13]:
import json

part3 = "/home/shivam/intern/repart/part-00003-06a43a4e-b6a6-45a0-a12d-0ee5869b345b-c000.json"

reviews4 = []
with open(part3) as p3:
    for line in p3:
        review = json.loads(line)
        reviews4.append(Review(review['reviewText'],review['overint']))

In [14]:
import json

part4 = "/home/shivam/intern/repart/part-00004-06a43a4e-b6a6-45a0-a12d-0ee5869b345b-c000.json"

reviews5 = []
with open(part4) as p4:
    for line in p4:
        review = json.loads(line)
        reviews5.append(Review(review['reviewText'],review['overint']))

In [88]:
import time
start_time = time.time()

from sklearn.model_selection import train_test_split
training,test = train_test_split(reviews1,test_size = 0.25,random_state = 40)

train_x = [x.text for x in training]
train_y = [x.sentiment for x in training]

test_x = [x.text for x in test]
test_y = [x.sentiment for x in test]

In [89]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)

In [90]:
from sklearn.tree import DecisionTreeClassifier
clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors,train_y)
x = clf_dec.predict(test_x_vectors)
print(x)
#from sklearn.linear_model import LogisticRegression
#clf_lg = LogisticRegression()
#clf_lg.fit(train_x_vectors,train_y)
#clf_lg.predict(test_x_vectors)

end_time = time.time()

print("--- %s seconds ---" % (end_time - start_time))

['Positive' 'Positive' 'Positive' ... 'Positive' 'Positive' 'Positive']
--- 78.44758582115173 seconds ---
